In [879]:
import math
import random

I. Random Input & Calculations

1. Random Input

In [880]:
def random_matrix(rows, cols, sparsity_percent):
	total_elements = rows * cols
	num_zeros = int(total_elements * sparsity_percent / 100)
	num_non_zeros = total_elements - num_zeros
	matrix_elements = [0] * num_zeros + [random.randint(1, 5) for _ in range(num_non_zeros)]
	random.shuffle(matrix_elements)
	sparse_matrix = []
	for i in range(rows):
		sparse_matrix.append(matrix_elements[i * cols:(i + 1) * cols])
	for row in sparse_matrix:
		if all(value == 0 for value in row):
			row[random.randint(0, cols - 1)] = random.randint(1, 9)
	return sparse_matrix

In [881]:
params = {
  "H_NUM_OF_ROWS": 10,
  "H_NUM_OF_COLS": 4,
  "W_NUM_OF_ROWS": 4,
  "W_NUM_OF_COLS": 4,
  "NUM_OF_NODES": 5,
  "A_DEPTH": 10
}

feature = random_matrix(params["H_NUM_OF_ROWS"], params["H_NUM_OF_COLS"],50)
weight = random_matrix(params["W_NUM_OF_ROWS"], params["W_NUM_OF_COLS"], 0)
a = [1 if i < params["A_DEPTH"] / 2 else 2 for i in range(params["A_DEPTH"])]
# a = random_matrix(1, params["A_DEPTH"], 0)[0]

2. SPMM

In [882]:
wh = [[0 for _ in range(len(weight[0]))] for _ in range(len(feature))]
for i in range(len(feature)):
  for j in range(len(weight[0])):
    for k in range(len(weight)):
      wh[i][j] += feature[i][k] * weight[k][j]

In [898]:
green = "\033[92m"
reset = "\033[0m"

def print_matrices_in_columns(feature, weight, wh):
    max_rows = max(len(feature), len(weight), len(wh))
    print("-" * 123)
    header = "\tWH\t\t\t\t\t\tFeature\t\t\t\t\t\tWeight"
    print(header)
    print("-" * 123)


    for i in range(max_rows):
        feature_row = feature[i] if i < len(feature) else [" "] * len(feature[0])
        weight_row = weight[i] if i < len(weight) else [" "] * len(weight[0])
        result_row = wh[i] if i < len(wh) else [" "] * len(wh[0])

        formatted_feature_row = " | ".join(f"{green}{str(value):<5}{reset}" if value != 0 else f"{str(value):<5}" for value in feature_row)
        formatted_weight_row = " | ".join(f"{str(value):<5}" for value in weight_row)
        formatted_result_row = " | ".join(f"{str(value):<5}" for value in result_row)

        print("| " + formatted_result_row + " | " + " " * 10 + " | " + formatted_feature_row + " | " + " " * 10 + " | " + formatted_weight_row + " | ")

    print("-" * 123)
print_matrices_in_columns(feature, weight, wh)

---------------------------------------------------------------------------------------------------------------------------
	WH						Feature						Weight
---------------------------------------------------------------------------------------------------------------------------
| 24    | 14    | 19    | 19    |            | 0     | 4     | 5     | 0     |            | 3     | 1     | 5     | 5     | 
| 10    | 5     | 12    | 14    |            | 2     | 0     | 0     | 1     |            | 1     | 1     | 1     | 1     | 
| 20    | 15    | 10    | 20    |            | 0     | 0     | 0     | 5     |            | 4     | 2     | 3     | 3     | 
| 11    | 9     | 7     | 11    |            | 0     | 3     | 0     | 2     |            | 4     | 3     | 2     | 4     | 
| 29    | 20    | 23    | 33    |            | 2     | 3     | 0     | 5     |            |       |       |       |       | 
| 28    | 17    | 24    | 30    |            | 2     | 2     | 2     | 3     |            |       

3. DMVM

In [884]:
midpoint = len(a) // 2
a_1 = a[:midpoint]
a_2 = a[midpoint:]

dmvm = []
coef = []

for index, row in enumerate(wh):
  sum = 0
  for i in range(len(row)):
    if index % params["NUM_OF_NODES"] == 0:
      sum += row[i] * a_1[i]
    else:
      sum += row[i] * a_2[i]
  dmvm.append(sum)
print("DMVM = ", dmvm)
for i in range(len(dmvm)):
  coef.append(dmvm[i] + dmvm[int(i / params["NUM_OF_NODES"]) * params["NUM_OF_NODES"]])
print("Coef = ", coef)

DMVM =  [76, 82, 130, 76, 210, 99, 26, 48, 198, 154]
Coef =  [152, 158, 206, 152, 286, 198, 125, 147, 297, 253]


II. Code Generator

1. GCSR structure

In [885]:
num_of_nodes = params["NUM_OF_NODES"]
col_idx, value, node_info = [], [], []

row_length = []
row_length_bits = math.ceil(math.log2(len(feature[0])))+1
num_of_nodes_bits = math.ceil(math.log2(num_of_nodes))
flag_bits = 1

for i, row in enumerate(feature):
  non_zero_elements = [(j, ele) for j, ele in enumerate(row) if ele != 0]
  col_idx.extend([idx for idx, _ in non_zero_elements])
  value.extend([val for _, val in non_zero_elements])

  row_len = len(non_zero_elements)
  row_length.append(row_len)
  flag = '1' if i % params["NUM_OF_NODES"]== 0 else '0'
  node_info_bin = (
    f"{row_len:0{row_length_bits}b}"
    f"{num_of_nodes:0{num_of_nodes_bits}b}"
    f"{flag}"
  )
  node_info.append(node_info_bin)

In [886]:
for row in feature:
	row_str = " | ".join(f"{green}{str(item):<5}{reset}" if item != 0 else f"{str(item):<5}" for item in row)
	print(row_str)

print("col_idx    = ", col_idx)
print("value      = ", value)
print("row_length = ", row_length)
print("node_info  = ", node_info)

0     | 4     | 5     | 0    
2     | 0     | 0     | 1    
0     | 0     | 0     | 5    
0     | 3     | 0     | 2    
2     | 3     | 0     | 5    
2     | 2     | 2     | 3    
0     | 0     | 0     | 1    
0     | 0     | 2     | 0    
0     | 0     | 5     | 3    
0     | 3     | 0     | 5    
col_idx    =  [1, 2, 0, 3, 3, 1, 3, 0, 1, 3, 0, 1, 2, 3, 3, 2, 2, 3, 1, 3]
value      =  [4, 5, 2, 1, 5, 3, 2, 2, 3, 5, 2, 2, 2, 3, 1, 2, 5, 3, 3, 5]
row_length =  [2, 2, 1, 2, 3, 4, 1, 1, 2, 2]
node_info  =  ['0101011', '0101010', '0011010', '0101010', '0111010', '1001011', '0011010', '0011010', '0101010', '0101010']


2. Code

In [887]:
def feature_code_generator():
  addr = 0
  script = "\tinitial begin\n"
  script += "\t\tH_col_idx_BRAM_ena = 1'b1;\n\t\tH_col_idx_BRAM_load_done = 1'b0;\n"
  script += "\t\tH_value_BRAM_ena = 1'b1;\n\t\tH_value_BRAM_load_done = 1'b0;\n\n"

  for col, val in zip(col_idx, value):
    script += f"\t\tH_col_idx_BRAM_din = {col};\n"
    script += f"\t\tH_value_BRAM_din = {val};\n"
    script += f"\t\tH_col_idx_BRAM_addra = {addr};\n"
    script += f"\t\tH_value_BRAM_addra = {addr};\n\n"
    script += "\t\t#20.01;\n"
    addr += 1

  script += "\t\tH_col_idx_BRAM_ena = 1'b0;\n\t\tH_col_idx_BRAM_load_done = 1'b1;\n"
  script += "\t\tH_value_BRAM_ena = 1'b0;\n\t\tH_value_BRAM_load_done = 1'b1;\n"
  script += "\tend\n\n"

  addr = 0
  script += "\tinitial begin\n"
  script += "\t\tH_node_info_BRAM_ena = 1'b1;\n\t\tH_node_info_BRAM_load_done = 1'b0;\n\n"
  for info in node_info:
    script += f"\t\tH_node_info_BRAM_din = {row_length_bits + num_of_nodes_bits + flag_bits}'b{info};\n"
    script += f"\t\tH_node_info_BRAM_addra = {addr};\n\n"
    script += "\t\t#20.01;\n"
    addr += 1

  script += "\t\tH_node_info_BRAM_ena = 1'b0;\n\t\tH_node_info_BRAM_load_done = 1'b1;\n"
  script += "\tend\n"
  return script

In [888]:
def weight_code_generator():
  addr = 0
  script = "\n\tinitial begin\n"
  script += "\t\tWeight_BRAM_ena = 1'b1;\n\t\tWeight_BRAM_load_done = 1'b0;\n\n"

  for row in weight:
    for val in row:
      script += f"\t\tWeight_BRAM_din = {val};\n"
      script += f"\t\tWeight_BRAM_addra = {addr};\n\n"
      script += "\t\t#20.01;\n"
      addr += 1

  script += "\t\tWeight_BRAM_ena = 1'b0;\n\t\tWeight_BRAM_load_done = 1'b1;\n"
  script += "\tend"
  return script

In [889]:
def a_code_generator():
  addr = 0
  script = "\n\tinitial begin\n"
  script += "\t\ta_BRAM_ena = 1'b1;\n\t\ta_BRAM_load_done = 1'b0;\n\n"

  for val in a:
    script += f"\t\ta_BRAM_din = {val};\n"
    script += f"\t\ta_BRAM_addra = {addr};\n\n"
    script += "\t\t#20.01;\n"
    addr += 1

  script += "\t\ta_BRAM_ena = 1'b0;\n\t\ta_BRAM_load_done = 1'b1;\n"
  script += "\tend"
  return script

III. Update code into testbench

In [890]:
filename = "D:/VLSI/Capstone/tb/top_tb.sv"

In [891]:
def update_input(filename, start_marker, end_marker, new_content):
  with open(filename, 'r') as file:
    file_data = file.read()

  start_index = file_data.find(start_marker)
  end_index = file_data.find(end_marker, start_index)

  updated_data = (
    file_data[:start_index + len(start_marker)] + "\n" + new_content + "\n\t" + file_data[end_index:]
  )
  with open(filename, 'w') as file:
    file.write(updated_data)

start_marker  = "// ---------------- Input ----------------"
end_marker    = "// ---------------------------------------"
new_content = feature_code_generator() + weight_code_generator() + a_code_generator()
update_input(filename, start_marker, end_marker, new_content)

In [892]:
def update_parameters(filename, parameters):
  with open(filename, 'r') as file:
    file_data = file.readlines()

  for i, line in enumerate(file_data):
    for param, new_value in parameters.items():
      found_param = "parameter " + param
      if found_param in line:
        file_data[i] = f"  parameter {param}\t\t\t= {new_value},\n"

  with open(filename, 'w') as file:
      file.writelines(file_data)

update_parameters(filename, params)

In [893]:
def write_list_to_file(file_path, data_list):
  with open(file_path, 'w') as file:
    for item in data_list:
      file.write(f"{item}\n")

write_list_to_file("inputs/col_idx.txt", col_idx)
write_list_to_file("inputs/value.txt", value)
write_list_to_file("inputs/node_info.txt", node_info)